## TOC:
* [Preparation](#preparation)
* [Basic Usages of Lanelet2 HDMap API](#lanelet2_basic)
* [Shortest Path Given by Lanelet2](#lanelet2_routing)
* [Global Path Planner with Lanelet2 Routing Submap](#lanelet_global_planner)

## Preparation <a class="anchor" id="preparation"></a>
We use Carla 0.9.13 as the simulation environment and conduct a series of experiments in map Town06. Before the start, we need to assure that Carla 0.9.13 is installed and the installation directory is set to system environment path. The following code will connect to the simulator and switch the map to Town06. We manually set the start position and the goal for futher experiments.

In [10]:
import carla
import numpy as np
import random
import sys
sys.path.append('../')

client = carla.Client('127.0.0.1', 2000)
world = client.load_world('Town06')
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]
start_point = (start_point.location.x, start_point.location.y, np.deg2rad(start_point.rotation.yaw))
end_point = spawn_points[10]
end_point = (end_point.location.x, end_point.location.y, np.deg2rad(end_point.rotation.yaw))


## Basic Usages of Lanelet2 HDMap API <a class="anchor" id="lanelet2_basic"></a>
In this section, we are going through the basic usages of lanelet2 HDMap. We frist parse the corresponding xml file and project the latitude and longitude on the x,y 2D plane. We then introduce the primitive elements of lanelet2 and try to illustrate them in the CARLA simulator. This includes
- points
- linestrings
- lanelets

In [11]:
import lanelet2
from lanelet2.projection import UtmProjector

lanelet2_town06 = '../resources/maps/Town06_hy.osm'
projector = UtmProjector(lanelet2.io.Origin(0., 0.))
loadedMap, load_errors = lanelet2.io.loadRobust(lanelet2_town06, projector) 
traffic_rules = lanelet2.traffic_rules.create(lanelet2.traffic_rules.Locations.Germany,
                                                lanelet2.traffic_rules.Participants.Vehicle)

In [3]:
## Draw Points (200m around the start position)
all_points = []
for point in loadedMap.pointLayer:    
    if np.linalg.norm((point.x - start_point[0], start_point[1] - point.y)) < 200:        
        world.debug.draw_string(carla.Location(point.x, -point.y, 0), '*',
                            color=carla.Color(0, 255, 0), life_time=15)   

In [4]:
from lanelet2.core import BasicPoint2d
## Draw the closest lanelet from the start position
world.debug.draw_string(carla.Location(start_point[0], start_point[1], 0.), 'start_pos', color=carla.Color(255, 0, 0), life_time=15)

closest_lanelet = lanelet2.geometry.findNearest(loadedMap.laneletLayer, BasicPoint2d(start_point[0], -start_point[1]), 1)[0][1]
for point in closest_lanelet.leftBound:
    world.debug.draw_string(carla.Location(point.x, -point.y, 0), 'l',
                            color=carla.Color(0, 255, 0), life_time=15) 

for point in closest_lanelet.rightBound:
    world.debug.draw_string(carla.Location(point.x, -point.y, 0), 'r',
                            color=carla.Color(0, 255, 0), life_time=15) 

for point in closest_lanelet.centerline:
    world.debug.draw_string(carla.Location(point.x, -point.y, 0), 'c',
                            color=carla.Color(0, 0, 255), life_time=15) 

## Shortest Path Given by Lanelet2 <a class="anchor" id="lanelet2_routing"></a>
The Lanelet2 library provides the method to plan a shortest path (a series of lanelets) through lanelets from the start to the goal. However, this lanelet path can not be utilized directly since it is not a trajectory.

In [5]:
start_lanelet = lanelet2.geometry.findNearest(loadedMap.laneletLayer, BasicPoint2d(start_point[0], -start_point[1]), 1)[0][1]
goal_lanelet = lanelet2.geometry.findNearest(loadedMap.laneletLayer, BasicPoint2d(end_point[0], -end_point[1]), 1)[0][1]
graph = lanelet2.routing.RoutingGraph(loadedMap, traffic_rules)
route = graph.getRoute(start_lanelet, goal_lanelet)
path = route.shortestPath()

for lanelet in path:
    for point in lanelet.leftBound:
        world.debug.draw_string(carla.Location(point.x, -point.y, 0), 'l',
                                color=carla.Color(0, 255, 0), life_time=15) 

    for point in lanelet.rightBound:
        world.debug.draw_string(carla.Location(point.x, -point.y, 0), 'r',
                                color=carla.Color(0, 255, 0), life_time=15) 

    for point in lanelet.centerline:
        world.debug.draw_string(carla.Location(point.x, -point.y, 0), 'c',
                                color=carla.Color(0, 0, 255), life_time=15) 

## Global Path Planner with Lanelet2 Routing Submap <a class="anchor" id="lanelet_global_planner"></a>

We now use our flood-fill based global planner to generate a global planning trajectory from the start to the goal. Dubins path is utilized to connect two lanelets. We create a solid line detector using KD-Tree to avoid crossover.
 We choose the trajectory that has the least lane-change distance.

In [12]:
from ISS.algorithms.planning.global_planner.lanelet2_planner import Lanelet2Planner
from ISS.algorithms.utils.vehicleutils.vehicleutils import CollisionChecker

def get_solid_checker(loadedMap):
    ## Get Solid Points...
    inset = set()
    solid_points = []
    solid_id = []
    for lanelet in loadedMap.laneletLayer:
        if 'subtype' not in lanelet.attributes or lanelet.attributes['subtype'] != 'road':
            continue
        left_lane = lanelet.leftBound
        right_lane = lanelet.rightBound        
        if "subtype" in left_lane.attributes and left_lane.attributes['subtype'] == 'solid' and left_lane.id not in inset:
            inset.add(left_lane.id)
            for point in left_lane:
                solid_id.append(left_lane.id)
                solid_points.append((point.x, -point.y))

        if "subtype" in right_lane.attributes and right_lane.attributes['subtype'] == 'solid' and right_lane.id not in inset:
            inset.add(right_lane.id)
            for point in right_lane:
                solid_id.append(right_lane.id)
                solid_points.append((point.x, -point.y))
    ##
    solid_checker = CollisionChecker(solid_points, 4.4, 2.2)
    return solid_checker

solid_checker = get_solid_checker(loadedMap)
planner = Lanelet2Planner(loadedMap, traffic_rules, solid_checker, reverse_y=True)
TURNING_RADIUS = 5.
traj = planner.plan(start_point, end_point, TURNING_RADIUS)
traj.downsample(0.1)
if traj != None:
    for point in traj.waypoints:
        world.debug.draw_string(carla.Location(point[0], point[1], 0), '*',
                    color=carla.Color(0, 255, 0), life_time=1200)        

In [13]:
from ISS.algorithms.localization.gt_carla import GroundTruthLocalizationCarla    
from ISS.algorithms.control.pid.pid import VehiclePIDController
import time

spawn_points = world.get_map().get_spawn_points()
spawn_point_ego = spawn_points[0]   
vehicle_bp = world.get_blueprint_library().find('vehicle.tesla.model3')
vehicle = world.spawn_actor(vehicle_bp, spawn_point_ego)    

controller = VehiclePIDController()
locator = GroundTruthLocalizationCarla(vehicle)
controller.set_traj(traj)
while True:
        time.sleep(0.01)
        vehicle_location = locator.run_step()
        state_cartesian = (vehicle_location.x, vehicle_location.y, vehicle_location.yaw, vehicle_location.velocity, vehicle_location.acceleration)        
        control_ = controller.run_step(vehicle_location, None)
        control = carla.VehicleControl()
        control.steer = control_.steer
        control.throttle = control_.throttle
        control.brake = control_.brake
        control.hand_brake = control_.hand_brake
        control.manual_gear_shift = control_.manual_gear_shift
        vehicle.apply_control(control)
        # world.tick()
        spec = world.get_spectator()
        spec_trans = vehicle.get_transform()
        ego_vehicle_location = vehicle.get_transform().location
        ego_vehicle_rotation = vehicle.get_transform().rotation
        offset = carla.Vector3D(x=-20.0, y=0, z=50)        
        left_rear_location = ego_vehicle_location + ego_vehicle_rotation.get_right_vector() * offset.y + \
                             ego_vehicle_rotation.get_forward_vector() * offset.x + \
                             ego_vehicle_rotation.get_up_vector() * offset.z
        rotation_reset = carla.Rotation(pitch=-45.0,
                                        yaw=ego_vehicle_rotation.yaw,
                                        roll=ego_vehicle_rotation.roll
                                        )
        spectator_transform = carla.Transform(left_rear_location, rotation_reset)
        spec.set_transform(spectator_transform)

KeyboardInterrupt: 